 PASO 1: Instalación de dependencias

In [1]:
# Instalar llama-cpp-python con soporte cuBLAS (GPU)
!pip uninstall -y llama-cpp-python -q
!pip install llama-cpp-python --extra-index-url=https://jllllll.github.io/llama-cpp-python-cuBLAS-wheels/avx2 -q

# Instalar Gradio para la interfaz conversacional
!pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 9.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00


PASO 2: Montar Google Drive y definir ruta del modelo GGUF

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Ruta al modelo Mistral cuantizado en formato .gguf
MODEL_PATH = "/content/drive/MyDrive/TFG/LLM/Modelos/mistral-7b-instruct-v0.2.Q4_K_M.gguf"

Mounted at /content/drive


PASO 3: Cargar el modelo Mistral 7B Instruct con llama-cpp-python

In [3]:
from llama_cpp import Llama

# Cargar modelo con parámetros óptimos
llm = Llama(
    model_path=MODEL_PATH,
    n_ctx=2048,          # Contexto suficiente para respuestas largas
    n_threads=4,         # Ajustable según CPU disponible
    n_gpu_layers=26      # Número de capas en GPU (ajustable a tu hardware)
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /content/drive/MyDrive/TFG/LLM/Modelos/mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_mo

PASO 4: Función para leer el JSON del análisis

In [4]:
import json

def leer_analisis_json(ruta="/content/drive/MyDrive/TFG/analisis_sistema.json"):
    try:
        with open(ruta, 'r') as f:
            datos = json.load(f)

        resumen = f"Último análisis: {datos['ultimo_analisis']}\n"
        resumen += f"Procesos analizados: {datos['procesos_analizados']}\n\n"

        sospechosos = [proc for proc in datos["resultados"] if proc["malware"]]
        if sospechosos:
            resumen += f"Se detectaron {len(sospechosos)} procesos sospechosos:\n"
            for p in sospechosos:
                resumen += f"- {p['nombre']} (prob: {p['probabilidad']})\n"
        else:
            resumen += "No se detectaron procesos sospechosos."

        return resumen
    except Exception as e:
        return f"Error al leer el análisis: {str(e)}"

 PASO 5: Función del chatbot (responde o consulta el JSON)

In [10]:
def responder(mensaje, _):
    mensaje_normalizado = mensaje.lower()

    if "análisis" in mensaje_normalizado or "procesos" in mensaje_normalizado or "malware" in mensaje_normalizado:
        respuesta = leer_analisis_json()
    else:
        prompt = f"{mensaje}\nAssistant:"
        salida = llm(prompt, max_tokens=256, stop=["</s>", "\n"])
        respuesta = salida["choices"][0]["text"].strip()

    return respuesta


PASO 6: Interfaz gráfica con Gradio

In [11]:
import gradio as gr

chatbot = gr.ChatInterface(
    fn=responder,
    title="KernelSight",
    description="Este asistente puede responder preguntas generales y consultar el resultado del análisis de malware del sistema.",
    examples=[
        "¿Como puedo defenderme frente a un ataque de Malware?",
        "Explícame qué es el phishing",
        "¿Hay procesos maliciosos en el sistema?",
        "Muéstrame el análisis de malware"
    ],
    cache_examples=False
)

chatbot.launch(debug=True)


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://09e1a1faf0e6e255ee.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Llama.generate: 1 prefix-match hit, remaining 454 prompt tokens to eval
llama_perf_context_print:        load time =    4232.49 ms
llama_perf_context_print: prompt eval time =  150500.44 ms /   454 tokens (  331.50 ms per token,     3.02 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  150503.22 ms /   455 tokens


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://09e1a1faf0e6e255ee.gradio.live
